<a href="https://colab.research.google.com/github/AugustoSavi/fine-tuning-studies/blob/main/Fine_Tune_BFC_Script_in_Llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning a programming language in Llama2
In this notebook, we will teach llama2 an programming language that it has no knowledge of.  BFC Scrip is a DSL.

This Notebook runs in Google Colab and is designed to run on a T4

# Setup the Model
The following section performs all the setup of the model.
This includes

- Installing any dependencies
- Setting any configuration
- Downloading the Base Model

## Install dependencies
In order to get started we need to install the appropriate dependencies

In [1]:
# install dependencies

# we use the latest version of transformers, peft, and accelerate
!pip install -q accelerate peft transformers

# install bitsandbytes for quantization
!pip install -q bitsandbytes

# install trl for the SFT library
!pip install -q trl

# we need sentencepiece for the llama2 slow tokenizer
!pip install sentencepiece

# we need einops, used by falcon-7b, llama-2 etc
# einops (einsteinops) is used to simplify tensorops by making them readable
!pip install -q -U einops

# we need to install datasets for our training dataset
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

## Settings
The following configures our settings for finetuning our model

In [2]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "AugustoSavi/bfc-test"

# Fine-tuned model name
new_model = "llama-2-7b-bfc-test"

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 20

## Download the base model
The following will download the base model, in this case the llama-2-7b-chat-hf model.

In [3]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    logging,
)

# load the quantized settings, we're doing 4 bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    # use the gpu
    device_map={"": 0}
)

# don't use the cache
model.config.use_cache = False

# Load the tokenizer from the model (llama2)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

# Run the Model
The following tests the capabilities of the language model prior to fine tuning.

In [4]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
#prompt = "What does the REM keyword stand for in OPL?"
#prompt = "What was the first implementation of the OPL programming language?"
#prompt = "Who were involved in the creation of OPL for Psion?"
#prompt = "Did Colly Myers create OPL?"
prompt = "escreva um hello world na linguagem bfc script "
#prompt = "Write a limerick as comments in the OPL programming language. "
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] escreva um hello world na linguagem bfc script  [/INST]  Claro! Aqui está um exemplo de "Hello World" em BFCLang:
 Hinweis: BFCLang é uma linguagem de programação orientada a objetos, então o exemplo aqui abaixo usará o formato de uma mensagem de "Hello World" como um objeto:
```
class HelloWorld {
  sayHello() {
    print("Hello, World!")
  }
}

const helloWorld = new HelloWorld()
helloWorld.sayHello()
```
Espero que isso ajude! Se você tiver alguma dúvida, por favor me avise.


# Train the Model
The following section is about taking your dataset and then finetuning the model

## Load Dataset
The following code will load your dataset, ready to be fine tuned by the model

In [13]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset(dataset_name, split="train")

train.jsonl:   0%|          | 0.00/27.4k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/139 [00:00<?, ? examples/s]

## Fine Tune the Model
The following section will take your dataset, and fine tune the model with it.

In [14]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,      # uses the number of epochs earlier
    per_device_train_batch_size=4,          # 4 seems reasonable
    gradient_accumulation_steps=2,          # 2 is fine, as we're a small batch
    optim="paged_adamw_32bit",              # default optimizer
    save_steps=0,                           # we're not gonna save
    logging_steps=10,                       # same value as used by Meta
    learning_rate=2e-4,                     # standard learning rate
    weight_decay=0.001,                     # standard weight decay 0.001
    fp16=False,                             # set to true for A100
    bf16=False,                             # set to true for A100
    max_grad_norm=0.3,                      # standard setting
    max_steps=-1,                           # needs to be -1, otherwise overrides epochs
    warmup_ratio=0.03,                      # standard warmup ratio
    group_by_length=True,                   # speeds up the training
    lr_scheduler_type="cosine",           # constant seems better than cosine
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,                # use our lora peft config
    #dataset_text_field="text",             # removed as no longer a valid parameter
    #max_seq_length=None,                    # no max sequence length
    tokenizer=tokenizer,                    # use the llama tokenizer
    args=training_arguments,                # use the training arguments
    #packing=False,                          # don't need packing
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

<ipython-input-14-f455750f5b51>:45: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/139 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/139 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/139 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/139 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/139 [00:00<?, ? examples/s]

{'loss': 3.6254, 'grad_norm': 0.7108919620513916, 'learning_rate': 0.00018181818181818183, 'mean_token_accuracy': 0.4419977396726608, 'epoch': 0.5714285714285714}
{'loss': 2.6424, 'grad_norm': 0.8119751214981079, 'learning_rate': 0.00019963094112117785, 'mean_token_accuracy': 0.5885008840184462, 'epoch': 1.1142857142857143}
{'loss': 1.5798, 'grad_norm': 0.6557250618934631, 'learning_rate': 0.00019835867895971014, 'mean_token_accuracy': 0.7594452440738678, 'epoch': 1.6857142857142857}
{'loss': 1.0369, 'grad_norm': 0.7978875041007996, 'learning_rate': 0.00019619024636010363, 'mean_token_accuracy': 0.822992183660206, 'epoch': 2.2285714285714286}
{'loss': 0.6233, 'grad_norm': 0.8315033316612244, 'learning_rate': 0.00019314540045221626, 'mean_token_accuracy': 0.8673614084720611, 'epoch': 2.8}
{'loss': 0.6835, 'grad_norm': 0.3970353305339813, 'learning_rate': 0.00018925188358598813, 'mean_token_accuracy': 0.8665971567756251, 'epoch': 3.342857142857143}
{'loss': 0.4782, 'grad_norm': 0.8758665

# Run the Model
The following runs the model post fine tune

In [17]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
#prompt = "What does the REM keyword stand for in OPL?"
#prompt = "What was the first implementation of the OPL programming language?"
#prompt = "Who were involved in the creation of OPL for Psion?"
#prompt = "Did Colly Myers create OPL?"
#prompt = "Write a hello world program in the OPL programming language, include an explanation of the code "
#prompt = "Write a limerick as comments in the OPL programming language. "
prompt = "como posso formatar uma data na linguagem bfc script"
#prompt = "Write a program that checks if a given year, is a leap year in OPL"
#prompt = "Write a hello world program in the OPL programming language. "
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] como posso formatar uma data na linguagem bfc script [/INST]  Para formatar uma data em Bfc script, você pode usar a função `formatarData(data, formatação)` onde `data` é a data a ser formatada e `formatação` é a formatação desejada.tamanho = 10
imprimir formatarData(data, tamanho)
imprimir formatarData(data, tamanho: 10)
imprimir formatarData(data, tamanho: {
    ano, mês, dia -> imprimir ano''- mês '-''dia
})
imprimir formatarData(data, tamanho: {
    ano, mês -> imprimir ano''- mês'- 'hoje
})
imprimir formatarData(data


In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir results/runs

In [ ]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

19965

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!huggingface-cli login

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mlabonne/llama-2-7b-miniguanaco/commit/c81a32fd0b4d39e252326e639d63e75aa68c9a4a', commit_message='Upload tokenizer', commit_description='', oid='c81a32fd0b4d39e252326e639d63e75aa68c9a4a', pr_url=None, pr_revision=None, pr_num=None)